In [4]:
%load_ext autoreload
%autoreload 2

# World of Warcraft Log Toy

This notebook toys with data generated from World of Warcraft (WoW) logs.

4 helper classes were created (in the wow folder):
- **wow.log.Record**
  - *Parses and stores an entry of the log*
  - Each line may have differente structure so more classes deriving from this one may be necessary.
  - Each record also represent an in game event.
- **wow.log.Encounter**
  - *A set of records that represents a fight agains a boss in the game*
  - Each starts with a ENCOUNTER_START event and go until an ENCOUNTER_END event is found.
- **wow.query.Query**
  - *An object to help create a chain of iterators to build queries (using filters and map like functions)*
- **wow.query.Predicate**
  - *A Collection of functions that will be applied to those iterators*

There are also another 2 modules in addition to both modules containing classes above:
- **ui**
  - contain ui elements in this notebook

- **fights**
  - contains scripts to analyze each fight (only nerzhul for now)


In [5]:
# Initialize
import wow.ui as ui

from wow.query import Query, Predicate
from wow.log import Record, Encounter

(z, f) = ui.import_file()

# Create Data Structures

In [10]:
encounters = Encounter.parse(f.content)

0  =>  20


# Detailed Analysis

In [13]:
# Define Encounter being Analysed
encSelect = ui.pick_encounter(encounters)

In [16]:
e = encounters[encSelect.value]

# Initial Analysis (Commented)

In [17]:
e.players

{('"Apolos-Azralon"', 'Player', 'Player-3209-05E7D17E'),
 ('"Bablowboy-Azralon"', 'Player', 'Player-3209-0B6675C9'),
 ('"Bilbos-Azralon"', 'Player', 'Player-3209-0A7915C2'),
 ('"Cotiamanca-Azralon"', 'Player', 'Player-3209-0B5C0896'),
 ('"Dii-Azralon"', 'Player', 'Player-3209-0B1A3D5B'),
 ('"Djøw-Azralon"', 'Player', 'Player-3209-0B560779'),
 ('"Flintti-Azralon"', 'Player', 'Player-3209-0B4848EE'),
 ('"Jaisonjjanke-Azralon"', 'Player', 'Player-3209-0B4B3CCB'),
 ('"Jathäl-Azralon"', 'Player', 'Player-3209-0B2A1849'),
 ('"Maulfurionx-Azralon"', 'Player', 'Player-3209-08518E93'),
 ('"Ryuudin-Azralon"', 'Player', 'Player-3209-0B560002'),
 ('"Shÿranui-Azralon"', 'Player', 'Player-3209-0A4BDD9A'),
 ('"Spartanovix-Azralon"', 'Player', 'Player-3209-0B588206'),
 ('"Theuzín-Azralon"', 'Player', 'Player-3209-0B5B6DAC'),
 ('"Thilokz-Azralon"', 'Player', 'Player-3209-0949ACB4'),
 ('"Vancartt-Azralon"', 'Player', 'Player-3209-0B6DCA27'),
 ('"Vracity-Azralon"', 'Player', 'Player-3209-0B5A533E'),
 ('"

{('"Burning Shade"', 'SPELL_AURA_APPLIED'),
 ('"Helm of Suffering"', 'SPELL_ABSORBED'),
 ('"Helm of Suffering"', 'SPELL_CAST_START'),
 ('"Helm of Suffering"', 'SPELL_CAST_SUCCESS'),
 ('"Helm of Suffering"', 'SPELL_DAMAGE'),
 ('"Orb of Torment"', 'SPELL_ABSORBED'),
 ('"Orb of Torment"', 'SPELL_AURA_APPLIED'),
 ('"Orb of Torment"', 'SPELL_AURA_REMOVED'),
 ('"Orb of Torment"', 'SPELL_CAST_START'),
 ('"Orb of Torment"', 'SPELL_CAST_SUCCESS'),
 ('"Orb of Torment"', 'SPELL_PERIODIC_DAMAGE'),
 ('"Orb of Torment"', 'SPELL_PERIODIC_MISSED')}

In [20]:
e.actor_actions('"Orb of Torment"')

[('"Eternal Torment"', 'SPELL_AURA_APPLIED', '"Orb of Torment"'),
 ('"Eternal Torment"', 'SPELL_CAST_SUCCESS', 'nil'),
 ('"Eternal Torment"', 'SPELL_AURA_APPLIED', '"Orb of Torment"'),
 ('"Eternal Torment"', 'SPELL_CAST_SUCCESS', 'nil'),
 ('"Torment"', 'SPELL_CAST_START', 'nil'),
 ('"Torment"', 'SPELL_CAST_SUCCESS', 'nil'),
 ('"Torment"', 'SPELL_AURA_APPLIED', '"Flintti-Azralon"'),
 ('"Torment"', 'SPELL_CAST_START', 'nil'),
 ('"Torment"', 'SPELL_AURA_APPLIED', '"Yapriesty-Azralon"'),
 ('"Torment"', 'SPELL_CAST_SUCCESS', 'nil'),
 ('"Torment"', 'SPELL_AURA_APPLIED', '"Jaisonjjanke-Azralon"'),
 ('"Torment"', 'SPELL_AURA_APPLIED', '"Dii-Azralon"'),
 ('"Torment"', 'SPELL_AURA_APPLIED', '"Cotiamanca-Azralon"'),
 ('"Torment"', 'SPELL_AURA_APPLIED', '"Apolos-Azralon"'),
 ('"Torment"', 'SPELL_ABSORBED', '"Flintti-Azralon"'),
 ('"Torment"', 'SPELL_ABSORBED', '"Flintti-Azralon"'),
 ('"Torment"', 'SPELL_PERIODIC_DAMAGE', '"Flintti-Azralon"'),
 ('"Torment"', 'SPELL_ABSORBED', '"Yapriesty-Azralon"')